In [1]:
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import cv2, 
import random
import json
import csv
import matplotlib.pyplot as plt

%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from tensorflow.keras import optimizers
import os


In [2]:
np.random.seed(20)


In [3]:
TRAIN_DIR = './cats_and_dogs_filtered/train/'
TEST_DIR = './cats_and_dogs_filtered/validation/'

ROWS = 150
COLS = 150
CHANNELS = 3

BATCH_SIZE=128

In [4]:
# Criando uma lista das imagens de treino totais e listas separadas para cada animal
original_train_images=[]
for i in os.listdir(TRAIN_DIR):
    #original_train_images.append(os.listdir(TRAIN_DIR+i))
    if i == 'dogs':
        train_dogs = os.listdir(TRAIN_DIR+i)
    if i == 'cats':
        train_cats = os.listdir(TRAIN_DIR+i)
    for images in os.listdir(TRAIN_DIR+i):
        original_train_images.append(f'{TRAIN_DIR}{i}/{images}')

random.shuffle(original_train_images)

#Separando a base de dados de treinamento em treinamento e validação
train_images = original_train_images[:1900]
validation_images = original_train_images[1900:]
print(len(train_images),len(validation_images))

1900 100


In [5]:
test_images=[]
for i in os.listdir(TEST_DIR):
    #print(TEST_DIR+i)
    for images in os.listdir(TEST_DIR+i):
        test_images.append(f'{TEST_DIR}{i}/{images}')

In [6]:
def plot_arr(arr):
    plt.figure()
    plt.imshow(image.array_to_img(arr))
    plt.show()

def plot(img):
    plt.figure()
    plt.imshow(img)
    plt.show()

In [7]:
def prep_data(images):
    count = len(images)
    X = np.ndarray((count, ROWS, COLS, CHANNELS), dtype=np.float32)
    y = np.zeros((count,), dtype=np.float32)
    print("Starting")
    for i, image_file in enumerate(images):
        img = image.load_img(image_file, target_size=(ROWS, COLS))
        X[i] = image.img_to_array(img)
        if 'dog.' in image_file:
            y[i] = 1.
        i+=1
        if i == count:
            print('Processed {} of {}'.format(i, count))
        else:
            if i%1000 == 0: print('Processed {} of {}'.format(i, count))
        
    return X, y


In [8]:
X_train, y_train = prep_data(train_images)

Starting
Processed 1000 of 1900
Processed 1900 of 1900


In [9]:
X_validation, y_validation = prep_data(validation_images)

Starting
Processed 100 of 100


In [10]:
train_datagen = image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

validation_datagen = image.ImageDataGenerator(rescale=1./255)


In [11]:
train_generator = train_datagen.flow(
    X_train,
    y_train,
    batch_size=BATCH_SIZE)

validation_generator = validation_datagen.flow(
    X_validation,
    y_validation,
    batch_size=BATCH_SIZE)

In [12]:
# def create_model():
#     model = Sequential()
#     model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(ROWS, COLS, CHANNELS)))
#     model.add(Conv2D(32, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(64, (3, 3), activation='relu'))
# #     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2,2)))
#     model.add(Conv2D(128, (3, 3), activation='relu'))
# #     model.add(Conv2D(128, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2,2)))
#     model.add(Conv2D(128, (3, 3), activation='relu'))
# #     model.add(MaxPooling2D(pool_size=(2,2)))
# #     model.add(Conv2D(256, (3, 3), activation='relu'))
# #     model.add(Conv2D(256, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2,2)))
#     # model.add(Dropout(0.5))

#     model.add(Flatten())
#     model.add(Dropout(0.5))
#     model.add(Dense(512, activation='relu'))
#     # model.add(Dense(1024, activation='relu'))
#     # model.add(Dense(1000, activation='relu'))
#     # model.add(Dropout(0.5))
#     model.add(Dense(1, activation='sigmoid'))
    
#     return model
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(ROWS, COLS, CHANNELS)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(64))

    model.add(Dense(1,activation='sigmoid'))
    return model


In [13]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 82944)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                5308480   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [14]:
model.compile(loss='binary_crossentropy',
             optimizer="adam",
             metrics=['accuracy'])

In [ ]:
train_steps = len(train_images)/BATCH_SIZE
validation_steps = len(validation_images)/BATCH_SIZE
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
#     steps_per_epoch=len(train_generator),
    epochs=11,
    validation_data=validation_generator,
    validation_steps=validation_steps,
#     validation_steps=len(validation_generator),
    verbose=2)


Epoch 1/11


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
model.save('dogs-v-cat-data-augmentation-04.h5')